In [2]:
import json
import pandas as pd
from datetime import datetime

In [1]:
LIST_OF_CHANNEL_IDS = [
# "UCsDmESjqNPukDmVnuneLrqw",
# "UCY6Ij8zOds0WJEeqCLOnqOQ",
# "UClN24S5-fc7LD0JJW0FdGWw",
# "UC3iNdSV_RQU7DHTGpqEW96w",
# "UC_52i6mlrzOZs_NEuSepzAg",
# "UCr3cBLTYmIK9kY0F_OdFWFQ",
# "UCV6g95OBbVtFmN9uiJzkFqQ",
# "UCgCKYs56-LKEPGQ99DzqQOg",
# "UCOwxx9VnEnlFKt5EB70KTzQ",
# "UC2Ds30pkifFVD0CE08wF50g",
# "UC7UGbBVrqLlq6CRxWWUmyKw",
# "UCn1XB-jvmd9fXMzhiA6IR0w",
# "UCFeqAfEuKm7lIg2ddQzh61A",
# "UCAlxwHEm1bWWYoe5VCSQYEg",
# "UCz7iJPVTBGX6DNO1RNI2Fcg",
# "UCOAUz4d1NUNxSHh_SOJtdGQ",
# "UCGGTAB19HlHEWPwwmxHsEKA",
# "UCOsATJw-IZgqGT8MFrHjKGg",
# "UCsKVP_4zQ877TEiH_Ih5yDQ",
# "UCOinp9hALPUcZeoL3Kmaviw",
# "UCGwu0nbY2wSkW8N-cghnLpA",
# "UCu6v4AdYxVH5fhfq9mi5llA",
# "UCj74rJ9Lgl3WTngq675wxKg",
# "UCNIuvl7V8zACPpTmmNIqP2A",
# "UCo8bcnLyZH8tBIH9V1mLgqQ",
# "UCR0O-1cvuPNxDosvSDLpWHg",
# "UCvUmwreRrbxeR1mbmojj8fg",
# "UCUKi4zY5ETSqrKAjTBgjM-g",
# "UCDQBZcjYKP1J1Nu-Y0_D37Q",
# "UCQ9HvHH-KRYHI5ynj2kbLwQ",
# "UCRfg0SWjIHm_h95e4V8X5og",
# "UCWPB0WpnMIy-g7zncwIhvQg",
#====moar
"UCj6CFdE3LSddaoszxq604TQ",
"UCXrtqz4fqf-OUEVpp2A1W0w",
"UChUg_Sd5i74SFoZH2XvagwQ",
"UC2hm5rD_IrfYRMfq5YQudgA",
"UC_nEHeUEVNY5ZYLRWg8KoZQ",
"UCrlzgjrKUa3AlrfSZTBG2fg",
"UCupWnlhB3PMW1w0CLfLCfug",
"UCsW0LA-ThH18OCrT9pa2zfQ",
"UCG-O5k7KKNPABULS20_MAMA",
"UCepPGz8AVCbggMl3BvboaBA",
"UCP21EOiKAkCGjv6kqS6UsZw",
"UC9z7EZAbkphEMg0SP7rw44A",
"UCZft8Ol-dMtguwam7bWItnw",
"UCYaxncRvOBu-vtdhxf6qciA",
"UCte3qx7A5OWTPSnNjV4oheg",
"UCRei8TBpt4r0WPZ7MkiKmVg",
"UC6bXfhbiX1PsSi_-pdQeHjg",
"UC_7lgWbCA_HZQkaxmmQgNoA",
"UCQO_C1sfUCpUzzV-gxQ-lxA",
"UCHw-JjzU80GKrGJwI1Uf1OQ",
"UCDEaY2XwwfKomQAJgGfehWw",
"UCI-92EMS3ZJ11gz-S_oyaSA",
"UC7Mk9RmlKT_x8FAsMOJn1cg",
"UC9i7x4EzTBlNvslu1wWEB8Q",
]

So I get the list of videos that a channel has. I don't yet have the specifics (aka metadata) on the videos yet because this API does not give me the information. What I have to do is filter this information to get a list of videos I need to throw at a different API to get the metadata. 

In [ ]:
def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [3]:
df = pd.read_json (r'searchListResponse_2.json')
channelIdToListOfVids = {} #channelId: []vids
backend_purposes = {}

#initialize the dicts
for channelId in LIST_OF_CHANNEL_IDS:
    channelIdToListOfVids[channelId] = []
    backend_purposes[channelId] = []

#youtuber
#0~9ish: items
#items: 0~46ish
test = df.loc[0, LIST_OF_CHANNEL_IDS[0]]
for index, midList in df.iterrows():  
    listOfLists = df.loc[index, LIST_OF_CHANNEL_IDS[index]]
    for indList in listOfLists:
        for item in indList['items']:
            channelId = item['snippet']['channelId']
            if 'videoId' in item['id'].keys():
                snippet=item['snippet']
                vidId = item['id']['videoId']
                publishTime = snippet['publishTime']
                vidTitle = snippet['title']
                vidDesc = snippet['description']

                vidInfo = {'vidId' : vidId,
                'publishTime': publishTime,
                'vidTitle': vidTitle}
                backend_purposes[channelId].append(vidId)
                channelIdToListOfVids[channelId].append(vidInfo)

count= 0
for i in backend_purposes.keys():
    count += len(backend_purposes[i])
print(count, " videos detected")

print(len(channelIdToListOfVids.keys()), " keys detected")
vidCont=0
for i in channelIdToListOfVids.keys():
    vidCont+=len(channelIdToListOfVids[i])
print(count, " videos detected")
# channelIdToListOfVids

1750  videos detected
24  keys detected
1750  videos detected


We now have the list of videoIds for each channel

In [4]:
listOfVideoIds=[]
for key, value in backend_purposes.items():
    listOfVideoIds+=value
with open('listOfVideoIds_2.json', 'w') as outfile:
    json.dump(listOfVideoIds, outfile)

After using this information in `main.py` to shoot at the API to get individual video metadata, I again filter out information specific to each video.

In [7]:
df = pd.read_json (r'videoListResponse.json')
matching_vidId_to_info={}
count = 0
debugMsg = []
for index, indList in df.iterrows():
    items = indList['items']
    for item in items:
        count+=1
        vidId = item['id']
        contentDetails= item['contentDetails']
        duration= contentDetails['duration']
        statistics = item['statistics']
        viewCount = statistics['viewCount']
        likeCount = statistics.get('likeCount')
        commentCount = statistics.get('commentCount')
        description = item['snippet']['description']
        if (likeCount == None or commentCount == None):
            print(vidId, "has", "likes" if likeCount == None else '', "comments" if commentCount == None else '', "disabled")
        topicCategories = []
        if item.get('topicDetails') != None:
            topicCategories = item.get('topicDetails').get('topicCategories')
        if True not in ['music' in strng.lower() for strng in topicCategories] or item.get('topicDetails') == None: #all videos that are not music-typed or just don't have a type
            matching_vidId_to_info[vidId] = {'duration': duration,
                            'viewCount': int(-1 if viewCount is None else viewCount),
                            'likeCount': int(-1 if likeCount is None else likeCount),
                            'commentCount': int(-1 if commentCount is None else commentCount),
                            'topicCategories': topicCategories,
                            'description': description}
        else: #all videos that are music-typed
            debugMsg.append(vidId)
print(count)
len(matching_vidId_to_info)
print("There were ", 3208-len(matching_vidId_to_info), " videos categorized as music-videos")


QQilWUln3F8 has  comments disabled
0eCXTYobXQk has  comments disabled
hpQQohcHk9Q has  comments disabled
fV15ba-dpPI has  comments disabled
Cb6F14AGrvI has  comments disabled
OmpK2tkKmNQ has likes  disabled
3222
There were  101  videos categorized as music-videos


In [8]:
#deepcopy
channelIdToListOfVids_copy = json.loads(json.dumps(channelIdToListOfVids))
debugMsg = []
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideosForEachChannel = channelIdToListOfVids_copy[channelId]
    for video in listOfVideosForEachChannel:
        additionalVideoMetadata= matching_vidId_to_info.get(video['vidId'])
        if additionalVideoMetadata:
            video.update(additionalVideoMetadata)
        else:
            debugMsg.append(video['vidId'])

print(sum(len(channelIdToListOfVids_copy[channelId]) for channelId in LIST_OF_CHANNEL_IDS))
print(len(debugMsg))
#some of these vids are private, duplicates. 

3223
103


In [9]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 14, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    #DO NOT RUN. IT'LL MODIFY THE _COPY
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

channelId_to_earliestVideoTimestamp_and_mostViewedVideo

{'UCsDmESjqNPukDmVnuneLrqw': {'firstUploadDate': '2009-12-07T21:13:28Z',
  'daysSinceFirstVideo': Timedelta('4389 days 00:00:00'),
  'mostViewedVideo': {'vidId': 'EcgkRp2IUsc',
   'publishTime': '2017-10-26T01:05:22Z',
   'vidTitle': 'My Hot Babysitter Got Me In Trouble',
   'duration': 'PT6M30S',
   'viewCount': 35958011.0,
   'likeCount': 840535.0,
   'commentCount': 75600.0,
   'topicCategories': ['https://en.wikipedia.org/wiki/Entertainment',
    'https://en.wikipedia.org/wiki/Film'],
   'description': "First Name Babysitter. Last Name Trouble. \nGET T-SHIRTS: http://www.itsalexclark.com \nSEE ME ON TOUR: http://www.itsalexclark.com/tour\n\nThanks to Shgurr and PamonSunset:\nhttps://www.youtube.com/channel/UCkQ3eqNNaFrBZ_WbnCIMN8w\n\nSubscribe: http://bit.ly/SubAlexClark | Get ItsAlexClark merch: http://itsalexclark.bigcartel.com/ \nWatch my latest video: https://youtube.com/watch?v=YIiBGDBN-Po&list=PLmh1WGagp73Jbndkw6nCz2JCm73jEp8gh\nHelp out the channel: https://Patreon.com/itsal

And now, I update the dictionary of each video specified by vidId, to update the video information with more metadata. Not only does the video info contain `vid publishedAt`, `publishTime`, `vidTitle`, and `vidDescription`, but it also contains `duration`, `viewCount`, `likeCount`, `commentCount`, and `topicCategories`

In [10]:
with open('masterSheet.json', 'w') as outfile:
    json.dump(channelIdToListOfVids_copy, outfile)

channelIdToListOfVidIds = {}
for channelId in LIST_OF_CHANNEL_IDS:
    vidInfoList = channelIdToListOfVids_copy[channelId]
    channelIdToListOfVidIds[channelId] = []
    for vidInfo in vidInfoList:
        channelIdToListOfVidIds[channelId].append(vidInfo['vidId'])

with open('masterSheetOfChannelIdToListOfVidIds.json', 'w') as outfile:
    json.dump(channelIdToListOfVidIds, outfile)

If I had a lot of resources like an actual GPU and memory, I'd probably analyze all the videos to find the average video to be representative of the channel, but since I don't, I want to take the most viewed video of each channel. This is a decision I made based on the... untestable hypothesis that the most-viewed video is the one that brought the channel the most subscriber count out of all of them. 

So I think taking the most-viewed video to represent the channel is an acceptable alternative. Thus, I will begin searching for 
1. the most viewed video by searching for `max(viewcount)` for each Channel
2. the first uploaded video by searching for `min(publishedAt)` for each channel because I need those stats as well. 

The result I get should be something along the lines of: `{ChannelId: 'xyz', earliestVideoPublishedDate: 'DateTime', mostViewedVideo : {vidInfo dictionary}}`

In [11]:
listOfVideoInfo = channelIdToListOfVids_copy["UCo8bcnLyZH8tBIH9V1mLgqQ"]
df = pd.DataFrame(list(listOfVideoInfo)) 
df

,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
0,CAb_bCtKuXg,2018-07-19T17:21:15Z,Life is Fun - Ft. Boyinaband (Official Music V...,NaN,NaN,NaN,NaN,NaN,NaN
1,cV542dRK3uk,2016-03-30T17:29:53Z,Work Stories (sooubway),PT7M21S,78753733.0,1129271.0,67830.0,[],"I was a sandwich ARTIST, okay? Because subways..."
2,ntHMsyr3R6w,2016-07-30T19:37:19Z,My Thoughts on Roommates,PT7M15S,88249252.0,1179279.0,90526.0,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Everyone come meet my (awesome) roommates. See...
3,2yFCyPX3kT0,2016-04-11T18:35:25Z,Annoying Customers,PT7M7S,94219677.0,1281249.0,71375.0,[],Here's part two of the subway stories. If any ...
4,_xBkp9fslQE,2018-03-26T19:14:47Z,My Mom&#39;s Cruel and Unusual Punishments,PT8M10S,45412248.0,1165210.0,149931.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",Thanks for raising me mom! You made me like th...
...,...,...,...,...,...,...,...,...,...
97,rj7Yo4AvCO8,2021-07-31T17:39:02Z,Catfish #shorts,PT14S,7345810.0,518199.0,7945.0,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,#shorts\nArt by:\nEmilee Dummer ➤ https://www....
98,0SY3yZId1sY,2021-07-17T15:41:13Z,A Villain with Disslexia #shorts,PT15S,4658611.0,222782.0,5108.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",#shorts\nArt by:\nEmilee Dummer ➤ https://www....
99,nwOB5HE8qiQ,2021-07-04T15:15:01Z,Lizard Cupid #shorts,PT17S,3809853.0,203630.0,4045.0,"[https://en.wikipedia.org/wiki/Entertainment, ...","Iguana be your Valentine #shorts \n(yes, this ..."
100,SVZC5lxyz_Q,2021-06-12T17:17:58Z,Dissection #shorts,PT14S,6899894.0,469600.0,7999.0,[https://en.wikipedia.org/wiki/Knowledge],Dissecting a frog is heartbreaking #shorts


So... creating a dataframe out of the info is quite trivial. Really easy. I also just noticed that some of these youtube channels are going to have music videos... which is not in the scope of my project right now. Gonna have to remove any videos with topicCategories of music themes.

In [12]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 14, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

channelId_to_earliestVideoTimestamp_and_mostViewedVideo

{'UCsDmESjqNPukDmVnuneLrqw': {'firstUploadDate': '2009-12-07T21:13:28Z',
  'daysSinceFirstVideo': Timedelta('4389 days 00:00:00'),
  'mostViewedVideo': {'vidId': 'EcgkRp2IUsc',
   'publishTime': '2017-10-26T01:05:22Z',
   'vidTitle': 'My Hot Babysitter Got Me In Trouble',
   'duration': 'PT6M30S',
   'viewCount': 35958011.0,
   'likeCount': 840535.0,
   'commentCount': 75600.0,
   'topicCategories': ['https://en.wikipedia.org/wiki/Entertainment',
    'https://en.wikipedia.org/wiki/Film'],
   'description': "First Name Babysitter. Last Name Trouble. \nGET T-SHIRTS: http://www.itsalexclark.com \nSEE ME ON TOUR: http://www.itsalexclark.com/tour\n\nThanks to Shgurr and PamonSunset:\nhttps://www.youtube.com/channel/UCkQ3eqNNaFrBZ_WbnCIMN8w\n\nSubscribe: http://bit.ly/SubAlexClark | Get ItsAlexClark merch: http://itsalexclark.bigcartel.com/ \nWatch my latest video: https://youtube.com/watch?v=YIiBGDBN-Po&list=PLmh1WGagp73Jbndkw6nCz2JCm73jEp8gh\nHelp out the channel: https://Patreon.com/itsal

In [50]:
#important. date ref is 12/11 because this is when I pulled this data. 

df = pd.read_json (r'channelListResponse.json')
REFERENCE_UTC_NOW= datetime(2021, 12, 12, 5, 2, 59, 166993)
channelInfo_master_df= pd.DataFrame()

for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']
    daysLive = pd.Timedelta(REFERENCE_UTC_NOW - datetime.strptime(publishedAt, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d').days

    data = {'channelName':title, 
    'channelId': channelId, 
    'daysLiveAsOfLaunch': daysLive,
    'daysSinceFirstVideo': channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days,
    'description': description,
    'viewCount': int(viewCount),
    'subscriberCount': int(subscriberCount),
    'videoCount': int(videoCount),
    'firstUploadDate' : channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['firstUploadDate'],
    'subscribersPerYearSinceFirstUpload' : float(int(subscriberCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    'viewCountPerYearSinceFirstUpload' : float(int(viewCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    }
    df_generated = pd.DataFrame(data, index=[index])
    channelInfo_master_df= channelInfo_master_df.append(df_generated)
channelInfo_master_df.head()

,channelName,channelId,daysLiveAsOfLaunch,daysSinceFirstVideo,description,viewCount,subscriberCount,videoCount,firstUploadDate,subscribersPerYearSinceFirstUpload,viewCountPerYearSinceFirstUpload
0,Alex Clark,UCsDmESjqNPukDmVnuneLrqw,4392,4389,Funny vids about my life. I'm thankful for th...,471903125,4280000,317,2009-12-07T21:13:28Z,355935.292777,3.924462e+07
1,Alex Meyers,UCY6Ij8zOds0WJEeqCLOnqOQ,2348,2341,I make cartoons and sometimes video essays abo...,472627239,2870000,256,2015-07-18T11:43:37Z,447479.709526,7.369028e+07
2,WowRightMeow,UClN24S5-fc7LD0JJW0FdGWw,1946,1809,"Yo! I'm Jamie, a dorky human from Wales. Did a...",29732357,537000,33,2016-12-31T15:08:07Z,108349.917081,5.999066e+06
3,Andrei Terbea,UC3iNdSV_RQU7DHTGpqEW96w,3147,3149,Doing animated commentary and solving crimes w...,292193694,2860000,136,2013-04-30T20:01:32Z,331502.064147,3.386812e+07
4,Billy But Better,UC_52i6mlrzOZs_NEuSepzAg,2805,1736,"Yes, it's a foot.\n\nSocial Medias:\nTwitter -...",12166120,260000,23,2017-03-14T02:31:14Z,54665.898618,2.557969e+06


In [40]:
with open("channelHolisticInfo.json", 'w') as f:
    json.dump(channelInfo_master_df.to_dict(), f)

`MostViewedVideoInfo` is showing up as NaN because it's of type `dict`. So need to just make sure I have access to the deets.

In [272]:
mostViewedVideo_master_dict = {}
for channelId in LIST_OF_CHANNEL_IDS:
    data = channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['mostViewedVideo']
    mostViewedVideo_master_dict[channelId]= data
    
mostViewedVideo_master_df = pd.DataFrame.from_dict(mostViewedVideo_master_dict).T
mostViewedVideo_master_df

,vidId,publishTime,vidTitle,vidDescription,duration,viewCount,likeCount,commentCount,topicCategories
UCsDmESjqNPukDmVnuneLrqw,EcgkRp2IUsc,2017-10-26T01:05:22Z,My Hot Babysitter Got Me In Trouble,First Name Babysitter. Last Name Trouble. GET ...,PT6M30S,35957532.0,840531.0,75609.0,"[https://en.wikipedia.org/wiki/Entertainment, ..."
UCY6Ij8zOds0WJEeqCLOnqOQ,kbCah6yhYRs,2019-06-11T11:00:07Z,Descendants doesn&#39;t make any sense...,Descendants doesn't make any sense animation w...,PT16M8S,9795510.0,262273.0,22454.0,"[https://en.wikipedia.org/wiki/Entertainment, ..."
UClN24S5-fc7LD0JJW0FdGWw,ewsGmhAjjjI,2018-10-09T20:50:50Z,I have APHANTASIA (and you may too...without r...,So...yeah. I'm an artist that can't see pictur...,PT12M54S,6023053.0,318934.0,71937.0,"[https://en.wikipedia.org/wiki/Hobby, https://..."
UC3iNdSV_RQU7DHTGpqEW96w,gA0bi-bFEYs,2018-06-22T19:20:36Z,How KSI Saved Logan Paul&#39;s Career,This KSI vs Logan Paul fight is interesting to...,PT6M48S,20316333.0,513661.0,95772.0,"[https://en.wikipedia.org/wiki/Entertainment, ..."
UC_52i6mlrzOZs_NEuSepzAg,so1_5hYUEE8,2017-09-20T22:12:54Z,Being a Boba Barista (Work Stories),Get it? “B”eing a “B”oba “B”arista “b”y “B”ill...,PT5M20S,2196394,58177,3759,[https://en.wikipedia.org/wiki/Lifestyle_(soci...
UCr3cBLTYmIK9kY0F_OdFWFQ,xa-4IAR_9Yw,2016-03-29T09:20:12Z,Casually Explained: Is She Into You?,Based on a true story. Dedicated to Jimmy. Unr...,PT2M31S,17105017,736340,20166,"[https://en.wikipedia.org/wiki/Entertainment, ..."
UCV6g95OBbVtFmN9uiJzkFqQ,I9uWUw1fxOY,2020-10-23T15:59:47Z,The Life of a Speedrunner,"ANOTHER DAY, ANOTHER CHANCE TO OPTIMIZE THIS R...",PT1M1S,19330823.0,832947.0,20369.0,[https://en.wikipedia.org/wiki/Video_game_cult...
UCgCKYs56-LKEPGQ99DzqQOg,W8P5ewPk9fM,2019-01-18T20:00:06Z,Why I Dont Want a Hot Wife (Ft. Emirichu),"This week, we talk about one of the things I g...",PT8M7S,2318751,115412,5847,[https://en.wikipedia.org/wiki/Entertainment]
UCOwxx9VnEnlFKt5EB70KTzQ,12Ne9n40tmw,2017-02-25T19:46:48Z,WHO&#39;S YOUR DADDY? ANIMATED with Pewdiepie ...,Animating Pewdiepie and Jacksepticeye's who's ...,PT2M26S,11703830,246733,6180,[]
UC2Ds30pkifFVD0CE08wF50g,Mv8OkBjySGQ,2017-09-15T00:08:53Z,My Super HOT English Teacher (Animation),"I have so many more stories, so subscribe and ...",PT8M5S,9247380,303609,26328,"[https://en.wikipedia.org/wiki/Entertainment, ..."


In [271]:
flatList = sum([mostViewedVideo_master_df.loc[channelId]['topicCategories'] for channelId in LIST_OF_CHANNEL_IDS], []) #monoid. apparently pretty bad performance
set(flatList)

{'https://en.wikipedia.org/wiki/Entertainment',
 'https://en.wikipedia.org/wiki/Film',
 'https://en.wikipedia.org/wiki/Hobby',
 'https://en.wikipedia.org/wiki/Knowledge',
 'https://en.wikipedia.org/wiki/Lifestyle_(sociology)',
 'https://en.wikipedia.org/wiki/Pet',
 'https://en.wikipedia.org/wiki/Role-playing_video_game',
 'https://en.wikipedia.org/wiki/Simulation_video_game',
 'https://en.wikipedia.org/wiki/Society',
 'https://en.wikipedia.org/wiki/Video_game_culture'}

So I need to remove... all the music-themed stuff. And I did. All the way up there.

Whew. And now... I need to gather the transcripts for the 33 videos above to run semantic analysis on it. There's probably an API for it...

In [278]:
channelId_to_mostViewedVidId = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = mostViewedVideo_master_dict[channelId]['vidId']
    channelId_to_mostViewedVidId[channelId] = mostViewedVidId

with open('channelId_to_mostViewedVidId.json', 'w') as outfile:
    json.dump(channelId_to_mostViewedVidId, outfile)

Ah. I am also interested in seeing when youtubers got their 100k milestone. Actually, the entire premise of this project was initially, me trying to find a link between their artstyle and historical sub count. 
Unfortunately, there isn't a way to get the historical subcount. Social blade does not manage subscriber count back in time, and I'm not sure if Youtube does, but even if it does, it's not available to the public. There's no APIs for the information. 

SO. I'm gonna go the cheap but mostly reliable route. Cheap because I'm going to be looking for `0k` in video descriptoins and titles, reliable because although self-reported, the youtuber has a growing fanbase.

In [32]:
count = 0

for channelId in LIST_OF_CHANNEL_IDS:
    listOfVidInfo = channelIdToListOfVids_copy[channelId]
    for vidInfo in listOfVidInfo:
        vidDescription = vidInfo['vidDescription']
        vidTitle = vidInfo['vidTitle']
        if vidDescription.find("0k") >= 0:
            count +=1
            print(vidDescription)
        if vidTitle.find("0k") >= 0 :
            count +=1
            print(vidTitle)
print(count)

To vote for me to make more cartoons click the vote button here(I'll win $250000k): http://bit.ly/grantplease Why are we totally ...
My Youtube Story  (30k Giveaway)
How I make my animation videos I really don't know what to say. I had completely given up on ever getting 50k, and 100k was just ...
How I make my videos (100k subscriber special)
MONSTER LEGENDS: https://monsterlegends.onelink.me/QSYm/AndreiTerbea Download before July 3rd and get 50k Food, 300k ...
This year has been crazy. We started with somewhere around 30k subscribers and we're finishing off the year with almost 1.3 ...
Yesterday we hit 200k subscribers and i made the super last minute decision to go live on Twitch and draw people in the chat as a ...
100k Subs + Anime North Recap
100k is a lot of nerds. A special thanks to Berd for Beh-ing in this video and also to Aaron at Nevercake for the nice poem and a ...
come see us draw like, everything Jazza's video: https://www.youtube.com/watch?v=0kI2JcvEDGw&ab_channel=Dra

and then I remembered this... figment of memory.
I think I saw a History section in wikitubia which tells you approximately when their subscriber milestone was.
alright. I guess we're doing this. This is actually going to be partially a webscraping project.
And this is quickly getting out of hand. I need to define the scope of this side project.

And... I should probably plan things out better. So... 
get channelId-> get their youtube videos by next token-> get video info. 

get transcripts
get mp4
get historical sub data.
get artstyle

Outcome. Let's limit it to 5. Use discretion when displaying data.
1. Historical subCount, normalized. Other easy stuff: average length of video (sum(Duration)/videoCount)
2. nlp (how positive/ negative is their content by transcripts)? word bubbles.
3. artstyle (colorized, less colorized, motion, shots)

============================================================================================================

***1. Dataframe data*** <br/>
By channelId:
* how long did it take them to get to 100k? What kind of videos did they publish before hitting 100k? I need chronological info.. 
--> 
    * average duration of video
    * subCount normalized
    * viewCount normalized

***2. NLP how positive/ negative is it?*** <br/>
By channelId:
* semantic analysis
* word bubble
* positivity/ negativity


***3. Artstyle*** <br/>
By channelId:
* color
* shots + shot length

***Conclusion***
1. Is there correlation between artstyle and sub count?
2. Is there a correlation between semantics + sub count?
3. Is there a common pattern that has gotten these animation youtubers to where they are today?